In [2]:
!python -V

Python 3.9.18


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

### Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [7]:
df_jan = pd.read_parquet("../../data/yellow_tripdata_2023-01.parquet")
no_columns = len(df_jan.columns)
print(f"Answer: {no_columns}")

Answer: 19


### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [8]:
df_jan["duration"] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan["duration"].apply(lambda td: td.total_seconds()/60)

standard_deviation = df_jan["duration"].std()

print(f"Answer: {standard_deviation}")

Answer: 42.59435124195458


### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [9]:
before = len(df_jan)
df_jan = df_jan[(df_jan.duration >=1) & (df_jan.duration <=60)]
after = len(df_jan)
fraction = round((after / before) * 100, 2)
print(f"Answer: {fraction} %")

Answer: 98.12 %


### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)   
Fit a dictionary vectorizer   
Get a feature matrix from it   
What's the dimensionality of this matrix (number of columns)?

In [10]:
df_jan[['PULocationID', 'DOLocationID']] = df_jan[['PULocationID', 'DOLocationID']].astype(str)  

dv = DictVectorizer()
train_dicts = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
dimension = X_train.shape[1]

print(f"Answer: {dimension}")

Answer: 515


### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable   
Calculate the RMSE of the model on the training data   
What's the RMSE on train?

In [11]:
target = 'duration'
y_train = df_jan[target].values
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
RMSE = root_mean_squared_error(y_train, y_pred)
print("Answer: ", RMSE)

Answer:  7.6492619533128225


### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [12]:
df_feb = pd.read_parquet("../../data/yellow_tripdata_2023-02.parquet")
df_feb["duration"] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration'] = df_feb["duration"].apply(lambda td: td.total_seconds()/60)

df_feb = df_feb[(df_feb.duration >=1) & (df_feb.duration <=60)]

df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype(str)  
validation_dicts = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_validation = dv.transform(validation_dicts)

y_validation = df_feb['duration'].values
y_prediction = model.predict(X_validation)
RMSE_validation = root_mean_squared_error(y_validation, y_prediction)
print("Answer: ", RMSE_validation)

Answer:  7.811816520976144
